# Teleportation Experiments

This notebook contains the 2D to 3D code teleportation experiments described in https://arxiv.org/abs/2510.07269.
The codes used are the bivariate bicycle/tricycle codes.

Env: Python 3.7

### imports

In [1]:
import numpy as np
import stim
import time
import sys
import itertools
import copy
from bposd.css import css_code

sys.path.append('../src/')
from TeleportationCircuit import build_logical_teleportation_circuit_2D_to_3D
from Decoders_SpaceTime import BPOSD_Decoder
from DistanceEst import DistanceEst_BPOSD
from beliefmatching import detector_error_model_to_check_matrices
from utilities import parmap
import multiprocessing as mp

In [2]:
import joblib
hx_dict_2d = joblib.load('../data/hx_dict_2d.pkl')
hz_dict_2d = joblib.load('../data/hz_dict_2d.pkl')
hx_dict_3d = joblib.load('../data/hx_dict_3d.pkl')
hz_dict_3d = joblib.load('../data/hz_dict_3d.pkl')
gamma_0_dict = joblib.load('../data/gamma_0_dict.pkl')
gamma_1_dict = joblib.load('../data/gamma_1_dict.pkl')
gamma_2_dict = joblib.load('../data/gamma_2_dict.pkl')
gamma_3_dict = joblib.load('../data/gamma_3_dict.pkl')

In [3]:
def create_and_simulate_2D_to_3D_teleportation_circuit(code_2D, code_3D, gamma_2, gamma_1, p, circuit_error_params, decoder_params, num_shots, psi, num_trials_decoder=1000, merged_schedule_2D=None, merged_schedule_3D=None):
    # 3. generate the teleportation circuit
    circuit = build_logical_teleportation_circuit_2D_to_3D(
        code_2D, code_3D, circuit_error_params, p, gamma_2, gamma_1, logical_psi = psi, merged_scheduling_2D=merged_schedule_2D, merged_scheduling_3D=merged_schedule_3D)

    # 4. set up sampler + DEM + get detector+logical values
    sampler = circuit.compile_detector_sampler()
    dem = circuit.detector_error_model(flatten_loops=True)
    
    num_shots_per_batch = 10_000
    # detector_vals, logical_vals = sampler.sample(shots=num_samples, separate_observables=True)
    det_chunks, log_chunks = [], []
    num_batches, leftover = num_shots//num_shots_per_batch, num_shots%num_shots_per_batch
    print(f"num shots {num_shots} | num batches {num_batches} | leftover {leftover}")
    for batch_i in range(num_batches+1):
        if batch_i%10 == 0:
            print(f"processing batch {batch_i}")
        if batch_i == num_batches:
            batch_detector_vals, batch_logical_vals = sampler.sample(leftover, separate_observables=True)
        else:
            batch_detector_vals, batch_logical_vals = sampler.sample(num_shots_per_batch, separate_observables=True)
        det_chunks.append(batch_detector_vals)
        log_chunks.append(batch_logical_vals)
    detector_vals = np.concatenate(det_chunks, axis=0)
    logical_vals  = np.concatenate(log_chunks,  axis=0)
    if len(detector_vals) != num_shots or len(logical_vals) != num_shots:
        print("incorrect number of shots")
        print(f"det len {len(detector_vals)} | log len {len(logical_vals)}")

    # 5. check circuit distance
    DemMatrices = detector_error_model_to_check_matrices(dem, allow_undecomposed_hyperedges=True)

    H = DemMatrices.check_matrix.toarray()
    L = DemMatrices.observables_matrix.toarray()
    channel_prob = DemMatrices.priors
    circuit_distance = DistanceEst_BPOSD(H, L, num_trials=num_trials_decoder)
    
    # 6. set up BPOSD decoder
    bposd_decoder = BPOSD_Decoder(H, channel_prob, decoder_params['max_iter'], decoder_params['bp_method'], 
                decoder_params['ms_scaling_factor'], decoder_params['osd_method'], decoder_params['osd_order'])

    # 7. get logical failure rate
    LFR = GetLFR(detector_vals, logical_vals, L, bposd_decoder)
    return circuit_distance, LFR

def GetLFR(detector_vals, logical_vals, L, decoder):
    def Run(i):
        synd1, logical1 = detector_vals[i], logical_vals[i]
        corr1 = decoder.decode(synd1)
        le1 = np.sum((logical1 + L@corr1)%2)
        return le1
    num_samples_run = len(detector_vals)
    start_time = time.time()
    if num_samples_run > 20000:
        batch_size = 10000
        N = int(num_samples_run/batch_size)
        Single_ERs = []
        for n_batch in range(N):
            ERs = parmap(Run, [n_batch*batch_size + i for i in range(batch_size)], nprocs = mp.cpu_count())
            Single_ERs += ERs
    else:
        Single_ERs = parmap(Run, [i for i in range(num_samples_run)], nprocs = mp.cpu_count())
    return np.average(Single_ERs) 

### set up codes

In [4]:
# set up 2D and 3D codes

codes_2D = {}
codes_3D = {}

for d in [3, 5]:
    code_2D = css_code(hx_dict_2d[f"d{d}_blp"], hz_dict_2d[f"d{d}_blp"])
    num_trials = 200
    dz = DistanceEst_BPOSD(code_2D.hx, code_2D.lx, num_trials)
    dx = DistanceEst_BPOSD(code_2D.hz, code_2D.lz, num_trials)
    code_2D.D = min(dz, dx)
    print("2D code BLP:", code_2D.code_params)
    codes_2D[f'd{d}_blp'] = code_2D

    code_3D = css_code(hx_dict_3d[f"d{d}_blp"], hz_dict_3d[f"d{d}_blp"])
    num_trials = 200
    dz = DistanceEst_BPOSD(code_3D.hx, code_3D.lx, num_trials)
    dx = DistanceEst_BPOSD(code_3D.hz, code_3D.lz, num_trials)
    code_3D.D = min(dz, dx)
    print("3D code BLP:", code_3D.code_params)
    codes_3D[f'd{d}_blp'] = code_3D

2D code BLP: (2,4)-[[18,2,3]]
3D code BLP: (4,6)-[[27,3,3]]
2D code BLP: (2,4)-[[54,2,6]]
3D code BLP: (4,6)-[[81,3,5]]


### set up experiments

In [5]:
num_points = 6
physical_error_rates = np.logspace(-3, -2, num=num_points)
decoder_params = {'max_iter':5, 'bp_method':'min_sum',
                  'ms_scaling_factor':0.4, 'osd_method':'osd_cs', 'osd_order': 11}
circuit_error_params = {"p_i": 0, "p_state_p": 1, "p_m": 1, "p_CX":1, "p_idling_gate": 0}

unnormalized_teleport_LFRs_dict = {code_id: [] for code_id in codes_3D.keys()}
normalized_teleport_LFRs_dict = {code_id: [] for code_id in codes_3D.keys()} # remember to normalize by k

In [6]:
num_samples_dict = {'d3_blp': [200000,100000,50000,20000,20000,20000],
                    'd5_blp': [200000,100000,50000,20000,20000,20000]}

In [7]:
num_shots_per_batch = 10_000
for code_id in ['d3_blp', 'd5_blp', 'd3_toric', 'd5_toric']:
# for code_id in ['d5_toric', 'd5_blp']:
    code_3D = codes_3D[code_id]
    code_2D = codes_2D[code_id]
    gamma_2 = gamma_2_dict[code_id]
    gamma_1 = gamma_1_dict[code_id]
    gamma_0 = gamma_0_dict[code_id]
    
    print("--------------------------------")
    print(f"CODES: {code_id} {code_2D.code_params} <> {code_3D.code_params}")
    for i, p in enumerate(physical_error_rates):
        num_shots = num_samples_dict[code_id][i]
        
        circ_dist, LFR = create_and_simulate_2D_to_3D_teleportation_circuit(code_2D,
                                                                            code_3D,
                                                                            gamma_2,
                                                                            gamma_1,
                                                                            p,
                                                                            circuit_error_params,
                                                                            decoder_params,
                                                                            num_shots,
                                                                            "+",
                                                                            500,
                                                                            None,
                                                                            None)
        normalized_LFR = 1-(1-LFR)**(1/(code_2D.K * code_2D.D))
        print(f"p={p:.10f} | num shots={num_shots} | LFR={LFR:5f} | normalized LFR={normalized_LFR}")
        unnormalized_teleport_LFRs_dict[code_id].append(LFR)
        normalized_teleport_LFRs_dict[code_id].append(normalized_LFR)

--------------------------------
CODES: d3_blp (2,4)-[[18,2,3]] <> (4,6)-[[27,3,3]]
num shots 200000 | num batches 20 | leftover 0
processing batch 0
processing batch 10


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

printed_code_params = {code_id: f"{code.code_params}".split('-')[1] for code_id, code in codes_3D.items()}

plt.rcParams.update({'font.size': 18})
fig = plt.figure()
fig.set_figwidth(14.4)
fig.set_figheight(10)

plt.plot(physical_error_rates, normalized_teleport_LFRs_dict['d3_blp'], marker='o', linestyle='-', color='C0', label=f"blp {printed_code_params['d3_blp']}", linewidth=3)
plt.plot(physical_error_rates, normalized_teleport_LFRs_dict['d5_blp'], marker='o', linestyle='-', color='C1', label=f"blp {printed_code_params['d5_blp']}", linewidth=3)

plt.xscale('log')
plt.yscale('log')
# plt.ylim(10e-6, 2*10e-2)

# Labels and title
plt.xlabel('Physical Error Rate')
plt.ylabel('Normalized Logical Failure Rate')
plt.title('Normalized Logical Failure Rate vs Physical Error Rate\n for Teleportation from 2D to 3D BLP Codes')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()

plt.tight_layout()
plt.show()